In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import os
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain import LLMChain
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    api_key=os.environ.get("GOOGLE_API_KEY")
)

In [4]:
search=TavilySearchResults(api_key=os.environ.get("TAVILY_API_KEY"))

C:\Users\suman\AppData\Local\Temp\ipykernel_19004\3621979532.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search=TavilySearchResults(api_key=os.environ.get("TAVILY_API_KEY"))


In [5]:
def purchase_link(query: str) -> str:
    """
    Function to get the purchase link for a tool or material.
    """
    search_results = search.run(query+ " buy online")
    if search_results:
        return search_results[0].link
    return "No purchase link found."

In [6]:
messages=[
    SystemMessage(content="You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc."),
    AIMessage(content="How can I assist you today?")
]

In [7]:
main_query="I want to hang a wall mirror in my living room."

In [8]:
messages.append(HumanMessage(content=main_query))
messages

[SystemMessage(content='You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to hang a wall mirror in my living room.', additional_kwargs={}, response_metadata={})]

In [9]:
messages.append(AIMessage(content="What problems are you running into with hanging the wall mirror?"))
messages.append(HumanMessage(content="I'm trying to hang a wall mirror but I'm not sure how to do it properly."))
messages.append(AIMessage(content="Describe it in more detail"))
messages.append(HumanMessage(content="I don't want to scratch the walls or break the glass. It has two hooks for attachments."))
messages.append(AIMessage(content="Do you have experience hanging a mirror?"))
messages.append(HumanMessage(content="Yes, I have hang a small to medium-sized mirror before."))
messages.append(AIMessage(content="Could you provide with a list of the tools and materials you have?"))
messages.append(HumanMessage(content="I have a hammer, nails, and a level."))

In [10]:
messages

[SystemMessage(content='You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to hang a wall mirror in my living room.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='What problems are you running into with hanging the wall mirror?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm trying to hang a wall mirror but I'm not sure how to do it properly.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Describe it in more detail', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I don't want to scratch the walls or break the glass. It has two hooks for attachments.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Do you have experience 

In [11]:
class Question(BaseModel):
    id: int
    text: str
    options: Optional[List[str]] = None


In [12]:
class QuestionsOutput(BaseModel):
    questions: List[Question]

In [13]:
parser = PydanticOutputParser(pydantic_object=QuestionsOutput)

In [14]:
template='''You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.
    You are given a previous conversation with the user. Your task is to generate as many questions that will help you understand the user's problem and the user objective better other than the provided ones.
    The questions should be concise and relevant to the user's task provided with options wherever possible. Only ask relevant questions and do not describe the task or provide any information. Just try to understand the user's problem better.
    Later there would be a missing tools and materials segment. So if there are any questions related to the tools and materials user has, you can ask them as well.
    But dont ask about any missing tools and materials.
    \n\n\n
    Previous conversation:\n{chat_history}
    \n\n\n
    {format_instructions}
    '''

In [15]:
question_prompts= PromptTemplate(
    template=template,
    input_variables=["chat_history"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [16]:
chain=LLMChain(
    llm=llm,
    prompt=question_prompts,
)
chat_history=messages

result=chain.run({"chat_history": chat_history})
output=parser.parse(result)
questions=output.questions

C:\Users\suman\AppData\Local\Temp\ipykernel_19004\2987848949.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(
C:\Users\suman\AppData\Local\Temp\ipykernel_19004\2987848949.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain.run({"chat_history": chat_history})


In [17]:
questions

[Question(id=1, text='What type of wall are you hanging the mirror on?', options=['Drywall', 'Plaster', 'Brick / Concrete', "I'm not sure"]),
 Question(id=2, text='What is the approximate weight of the mirror?', options=['Light (under 10 lbs / 5 kg)', 'Medium (10-25 lbs / 5-11 kg)', 'Heavy (over 25 lbs / 11 kg)']),
 Question(id=3, text='What are the approximate dimensions (width and height) of the mirror?', options=None),
 Question(id=4, text='Are the two hooks on the back connected by a wire?', options=['Yes, there is a wire between them', 'No, they are separate hooks (e.g., D-rings)']),
 Question(id=5, text='Do you have a tape measure?', options=['Yes', 'No']),
 Question(id=6, text='Do you have a stud finder to locate the support beams in the wall?', options=['Yes', 'No']),
 Question(id=7, text="Is the mirror's frame made of a heavy material like wood or metal?", options=["Yes, it's a heavy frame", "No, it's a light frame (e.g., plastic) or it's frameless"])]

In [18]:
len(questions)

7

In [19]:
messages.append(AIMessage(content="Here are some questions to help me understand your task better:"))
for q in questions:
    if q.options:
        options=""
        for q_option in q.options:
            options += f"{q_option}\n"
        messages.append(AIMessage(content=f"Q{q.id}: {q.text}\nOptions:\n{options}"))
    else:
        messages.append(AIMessage(content=f"Q{q.id}: {q.text}"))

    print(messages[-1].content)
    user_query=input("Your response: ")
    messages.append(HumanMessage(content=user_query))

Q1: What type of wall are you hanging the mirror on?
Options:
Drywall
Plaster
Brick / Concrete
I'm not sure

Q2: What is the approximate weight of the mirror?
Options:
Light (under 10 lbs / 5 kg)
Medium (10-25 lbs / 5-11 kg)
Heavy (over 25 lbs / 11 kg)

Q3: What are the approximate dimensions (width and height) of the mirror?
Q4: Are the two hooks on the back connected by a wire?
Options:
Yes, there is a wire between them
No, they are separate hooks (e.g., D-rings)

Q5: Do you have a tape measure?
Options:
Yes
No

Q6: Do you have a stud finder to locate the support beams in the wall?
Options:
Yes
No

Q7: Is the mirror's frame made of a heavy material like wood or metal?
Options:
Yes, it's a heavy frame
No, it's a light frame (e.g., plastic) or it's frameless



In [20]:
messages

[SystemMessage(content='You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to hang a wall mirror in my living room.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='What problems are you running into with hanging the wall mirror?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm trying to hang a wall mirror but I'm not sure how to do it properly.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Describe it in more detail', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I don't want to scratch the walls or break the glass. It has two hooks for attachments.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Do you have experience 

In [21]:
class ToolsRequired(BaseModel):
    tool_name: str=Field(description="Name of the missing tools required for the task")
    description: str=Field(description="Short Description of the missing tools required for the task")
    

In [22]:
class ToolsOutput(BaseModel):
    tools: List[ToolsRequired]

In [23]:
tools_parser=PydanticOutputParser(pydantic_object=ToolsOutput)

In [24]:
tools_prompt=PromptTemplate(
    template='''You are a helpful tools and materials expert assistant who suggests the tools and materials required for a particular task.
    Based on the conversation, you will need to suggest the tools and materials required for the task.
    
    Also analyze the user experience, the tools and materials they have and all the details of the tasks provided in the conversation.
    Using these informations, suggest the most basic to advanced tools and materials required for the task.
    \n\n\n
    Previous conversation:\n{chat_history}
    \n\n\n
    {format_instructions}''',
    input_variables=["chat_history"],
    partial_variables={"format_instructions": tools_parser.get_format_instructions()}
)

In [25]:
tools_llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=os.environ.get("GOOGLE_API_KEY")
)

In [26]:
chain=tools_prompt | tools_llm | tools_parser

In [27]:
tools_suggestion=chain.invoke({"chat_history": messages})

In [30]:
messages.append(AIMessage(content="Here are the tools and materials required for your task:"))
for items in tools_suggestion.tools:
    search_query="Can you provide me the purchase link of the "+ items.tool_name+" from Amazon?"
    results=search.invoke(search_query)
    if len(results):
        print(f"Tool name : {items.tool_name}\nTool description : {items.description}\nLink Title: {results[0].get('title')}\nURL: {results[0].get('url')}")

Tool name : Stud Finder
Tool description : Essential for locating wooden studs behind your drywall. Hanging a heavy mirror (over 25 lbs) directly into a stud provides the most secure attachment point, significantly reducing the risk of the mirror falling and damaging the wall or breaking.
Link Title: Zircon: Stud Finders - Amazon.com
URL: https://www.amazon.com/stores/Zircon/page/1A422F09-114D-4A08-80F7-D9B5DA29FC83
Tool name : Power Drill
Tool description : Required for drilling pilot holes if you locate a stud, or for installing heavy-duty drywall anchors (such as toggle bolts or molly bolts) if a stud is not available at your desired hanging location. Nails alone are not sufficient for a heavy mirror on drywall.
Link Title: Power Drill Cordless - Amazon.com
URL: https://www.amazon.com/Power-Drills-Cordless-Tools/s?keywords=Power+Drills&rh=n%3A552794%2Cp_n_feature_four_browse-bin%3A9060577011&c=ts&ts_id=552794
Tool name : Drill Bit Set
Tool description : A set of various drill bits w

In [35]:
for items in tools_suggestion.tools:
    messages.append(AIMessage(content=f"Tool Name: {items.tool_name}\nDescription: {items.description}"))

In [36]:
messages

[SystemMessage(content='You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to hang a wall mirror in my living room.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='What problems are you running into with hanging the wall mirror?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm trying to hang a wall mirror but I'm not sure how to do it properly.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Describe it in more detail', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I don't want to scratch the walls or break the glass. It has two hooks for attachments.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Do you have experience 

In [37]:
class StepsRequired(BaseModel):
    step_id: int=Field(description="Step number for the task")
    step: str=Field(description="Step by step to-dos to complete the task")
    description: str=Field(description="Description of the step required to complete the task")
    risk_factors: str=Field(description="Risk factors involved in the step to complete the task")
    safety_measures: str=Field(description="Safety measures to take while performing the step to complete the task")

In [38]:
class StepsOutput(BaseModel):
    steps: List[StepsRequired]

In [39]:
steps_parser=PydanticOutputParser(pydantic_object=StepsOutput)

In [40]:
steps_prompt=PromptTemplate(
    template='''You are a helpful expert assistant for DIY tasks who suggests the step by step to-dos to complete the task.
    Based on the conversation, you will need to suggest the step by step approaches along with their description (how to use the tools for the task) and safety measures.
    Analyze the whole conversation and suggest the steps required to complete the task in an easy and simple way.\n\n\n
    Previous conversation:\n{chat_history}
    \n\n\n
    {format_instructions}''',
    input_variables=["chat_history"],
    partial_variables={"format_instructions": steps_parser.get_format_instructions()}
)

In [41]:
steps_chain=steps_prompt | llm | steps_parser

In [42]:
steps=steps_chain.invoke({"chat_history": messages})
print(steps)

steps=[StepsRequired(step_id=1, step='Plan and Measure Your Placement', description="First, decide on the perfect spot for your mirror. A general guideline is to hang it so the center is at eye level, which is typically about 60 inches (152 cm) from the floor. Have a helper hold the mirror against the wall to visualize the final position. Once you're happy with the placement, use your tape measure and a pencil to make a light, small mark on the wall indicating the top center of the mirror frame.", risk_factors='Incorrect placement can result in an off-center or poorly positioned mirror, requiring you to patch holes and start over. Permanent marks can damage the wall paint.', safety_measures='Use a pencil for marks, as they are easily erased. Confirm the placement with a helper before marking. Measure twice to ensure accuracy.'), StepsRequired(step_id=2, step='Locate the Wall Studs', description='Since your mirror is heavy, hanging it on wall studs is the safest option. Turn on your stu

In [44]:
for step in steps.steps:
    print(f"Step {step.step_id}:{step.step}\nDescription: {step.description}\nRisk Factors: {step.risk_factors}\nSafety Measures: {step.safety_measures}\n")

Step 1:Plan and Measure Your Placement
Description: First, decide on the perfect spot for your mirror. A general guideline is to hang it so the center is at eye level, which is typically about 60 inches (152 cm) from the floor. Have a helper hold the mirror against the wall to visualize the final position. Once you're happy with the placement, use your tape measure and a pencil to make a light, small mark on the wall indicating the top center of the mirror frame.
Risk Factors: Incorrect placement can result in an off-center or poorly positioned mirror, requiring you to patch holes and start over. Permanent marks can damage the wall paint.
Safety Measures: Use a pencil for marks, as they are easily erased. Confirm the placement with a helper before marking. Measure twice to ensure accuracy.

Step 2:Locate the Wall Studs
Description: Since your mirror is heavy, hanging it on wall studs is the safest option. Turn on your stud finder and calibrate it against the wall according to its instr

In [46]:
messages.append(HumanMessage(content="Provide me the step by step approaches to complete the task along with their description and safety measures."))
messages.append(AIMessage(content="Here are the step by step approaches to complete your task along with their description and safety measures:"))

In [47]:
for step in steps.steps:
    messages.append(AIMessage(content=f"Step {step.step_id}:{step.step}\nDescription: {step.description}\nRisk Factors: {step.risk_factors}\nSafety Measures: {step.safety_measures}\n"))
    break


In [48]:
messages

[SystemMessage(content='You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to hang a wall mirror in my living room.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='What problems are you running into with hanging the wall mirror?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm trying to hang a wall mirror but I'm not sure how to do it properly.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Describe it in more detail', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I don't want to scratch the walls or break the glass. It has two hooks for attachments.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Do you have experience 

In [50]:
chat_template=PromptTemplate(
    template='''You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.
    You are given a previous conversation with the user. Your task is to generate a concise and relevant response to the user's query to solve their doubts or suggests any alternates as per user query.
    Like a chatbot, make the user confident that you are here to help them with their task and provide them with the best possible solution.
    Dont jump to next steps and just focus on solving the doubts of the current query.
    \n\n\n
    Previous conversation:\n{chat_history}
    \n\n\n
    Doubts and queries:\n{query}\n''',
    input_variables=["chat_history", "query"],
)

In [51]:
chat_parser=StrOutputParser()

In [52]:
chat_llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=os.environ.get("GOOGLE_API_KEY")
)

In [53]:
chat_chain=chat_template | chat_llm | chat_parser
resp=chat_chain.invoke({"chat_history": messages, "query": "I dont have anyone to hold the mirror while I am hanging it. What should I do?"})

In [54]:
print(resp)

No worries at all! You can definitely plan and measure your mirror's placement even without a helper.

Instead of having someone hold the mirror, you can use your **painter's tape** to outline the mirror's dimensions on the wall. Here's how:

1.  **Measure the mirror's total height and width** with your tape measure.
2.  **Use your painter's tape to create an outline of the mirror on the wall** at your desired height and width. This will give you a visual representation of the mirror's footprint.
3.  **Step back and assess** if this placement looks good in the room. You can easily adjust the tape until you're happy with the position.
4.  Once satisfied, you can then make your light pencil mark for the top center of the mirror frame within this tape outline.

This method allows you to visualize and confirm the perfect spot for your mirror independently, ensuring precise placement without any risk to your walls or the mirror itself. I'm here to guide you through each step to make sure yo

In [55]:
messages.append(HumanMessage(content="I dont have anyone to hold the mirror while I am hanging it. What should I do?"))
messages.append(AIMessage(content=resp))

In [56]:
messages

[SystemMessage(content='You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to hang a wall mirror in my living room.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='What problems are you running into with hanging the wall mirror?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm trying to hang a wall mirror but I'm not sure how to do it properly.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Describe it in more detail', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I don't want to scratch the walls or break the glass. It has two hooks for attachments.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Do you have experience 

In [57]:
messages.append(HumanMessage(content="Its great and everything is perfect."))

In [86]:
for step in steps.steps:
    print(step.step_id)
    if step.step_id == 6:
        messages.append(AIMessage(content=f"Step {step.step_id}:{step.step}\nDescription: {step.description}\nRisk Factors: {step.risk_factors}\nSafety Measures: {step.safety_measures}\n"))
        break

1
2
3
4
5
6


In [87]:
messages

[SystemMessage(content='You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to hang a wall mirror in my living room.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='What problems are you running into with hanging the wall mirror?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm trying to hang a wall mirror but I'm not sure how to do it properly.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Describe it in more detail', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I don't want to scratch the walls or break the glass. It has two hooks for attachments.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Do you have experience 

In [66]:
doubts="I am not confident about using the stud finder properly. Can you explain how to use it?"
resp=chat_chain.invoke({"chat_history": messages, "query": doubts})
print(resp)

Absolutely, it's completely normal to feel unsure about using a new tool like a stud finder, especially when a heavy mirror is involved! I'm here to make sure you feel confident with every step.

Here's a simple breakdown of how to use a stud finder properly:

1.  **Turn it On and Calibrate:** Most stud finders have a power button. Once on, some models require a quick calibration. This usually involves placing it flat against the wall where you *know* there's no stud (e.g., in the middle of a wall section) and pressing a calibrate button until it beeps or lights up, indicating it's ready. This sets its baseline.
2.  **Place and Slide:** Place the stud finder flat against the wall at the approximate height where your mirror's hanging point will be. Slowly slide it horizontally across the wall in one continuous motion.
3.  **Look/Listen for Indicators:** As you slide, the stud finder will typically light up, beep, or display an arrow when it detects a stud. Many will indicate the *edge* 

In [67]:
messages.append(HumanMessage(content=doubts))
messages.append(AIMessage(content=resp))

In [69]:
messages.append(HumanMessage(content="Yeah!! I am done with this step and I am ready to hang the mirror now."))

In [72]:
doubts="I am a bit confused about the measurements. Can you help me with that?"
resp=chat_chain.invoke({"chat_history": messages, "query": doubts})
print(resp)

Absolutely, it's common for measurements to feel a bit tricky, especially when you want to get it just right! I'm happy to clarify Step 3 for you to make sure you're confident before moving forward.

Step 3 involves two main types of measurements to ensure your heavy mirror hangs securely and perfectly level:

1.  **Finding the "Drop" Distance (Measurement on the Mirror):**
    *   **Purpose:** This tells you how far *down* from the very top edge of your mirror frame the actual hanging point (the taut wire) will be.
    *   **How to do it:**
        *   Hold your mirror or lay it face down.
        *   Take the hanging wire on the back and pull it upwards and tight, just as it would be if it were hanging on a hook.
        *   Using your tape measure, measure the distance from the *very top edge of the mirror's frame* straight down to the *highest point of the taut wire* (the peak of the triangle the wire forms when pulled tight).
        *   Write this measurement down carefully. This

In [73]:
messages.append(HumanMessage(content=doubts))
messages.append(AIMessage(content=resp))
messages.append(HumanMessage(content="Got it and I'm done with the markings and the measurements."))

In [77]:
doubts="I am not sure about the anchor type and its size to use for the wall. Can you help me with that?"
resp=chat_chain.invoke({"chat_history": messages, "query": doubts})
print(resp)

Absolutely, it's crucial to choose the right anchor type and size for a heavy mirror on drywall – that's a smart question to ask! I'm here to ensure you select the most secure option.

For a mirror weighing over 25 lbs (11 kg) on drywall, you **must use heavy-duty drywall anchors**. Standard nails or small plastic anchors will not be sufficient and could lead to the mirror falling.

Here are the most reliable types and how to think about sizing:

1.  **Toggle Bolts (Butterfly Anchors):**
    *   **Description:** These are excellent for very heavy loads. They require a larger hole, but once inserted, a metal "wing" springs open behind the drywall, creating a very strong grip.
    *   **Sizing:** They come in various sizes (e.g., 1/8 inch, 3/16 inch, 1/4 inch). The size refers to the bolt diameter. **Always check the packaging for the weight rating.** For your heavy mirror, look for toggle bolts rated for at least 50 lbs (or even 75-100 lbs for extra peace of mind) *per anchor*. Since yo

In [78]:
messages.append(HumanMessage(content=doubts))
messages.append(AIMessage(content=resp))
doubts="I think I should use a heavy-duty self-drilling drywall anchor as suggested for the wall. Is that correct as per the conditions?"
resp=chat_chain.invoke({"chat_history": messages, "query": doubts})
print(resp)

Yes, that's absolutely correct and a smart choice! Given that your mirror is heavy (over 25 lbs) and you're hanging it on drywall, a **heavy-duty self-drilling drywall anchor** is indeed a very suitable and secure option, especially if you're not able to drill directly into a wall stud at your desired location.

These types of anchors are specifically designed to provide strong support in drywall for heavier items. Just make sure to:
*   **Check the weight rating on the anchor's packaging** to ensure it can safely support at least double the weight of your mirror.
*   **Follow the manufacturer's instructions** for installation, including the correct drill bit size if needed.

You're on the right track to ensuring your mirror is hung safely and securely!


In [79]:
messages.append(HumanMessage(content=doubts))
messages.append(AIMessage(content=resp))
messages.append(HumanMessage(content="I have completed this step and ready for the next."))

In [83]:
doubts="I am working alone and I am not sure how to hold the mirror while I am hanging it. Can you help me with that?"
resp=chat_chain.invoke({"chat_history": messages, "query": doubts})
print(resp)

No worries at all! Hanging a heavy mirror by yourself can be done safely with a few smart adjustments, especially since you've already got your precise markings and heavy-duty anchors in place. You don't necessarily need a helper for this final step, though it always makes it easier.

Here's how you can safely hang the heavy mirror by yourself:

1.  **Create a Temporary Support Ledge:**
    *   Find a sturdy, stable object that you can place directly below where the bottom of your mirror will rest on the wall. This could be a small step stool, a stack of strong books, or a sturdy box.
    *   The top of this object should be just a few inches *below* the final resting height of the mirror's bottom edge. This will allow the mirror to lean against it, taking most of the weight off your arms while you align the hooks.

2.  **Prepare the Mirror:**
    *   Put on your **work gloves** for a better grip and protection.
    *   Carefully lift the mirror. Instead of trying to lift it high, focu

In [84]:
messages.append(HumanMessage(content=doubts))
messages.append(AIMessage(content=resp))
messages.append(HumanMessage(content="Yeah I am done with this step."))

In [88]:
messages.append(HumanMessage(content="Thank you for your help! I have completed the task successfully and it looks great."))

In [89]:
messages

[SystemMessage(content='You are a helpful home expert assistant who assists users with DIY home tasks like hanging wall mirrors, painting walls, installing shelves, etc.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to hang a wall mirror in my living room.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='What problems are you running into with hanging the wall mirror?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm trying to hang a wall mirror but I'm not sure how to do it properly.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Describe it in more detail', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I don't want to scratch the walls or break the glass. It has two hooks for attachments.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Do you have experience 

In [ ]:
for msg in messages:
    if isinstance(msg, HumanMessage):
        print(f"YOU : {msg.content}")
    else:
        if 
        print(f"ASSISTANT : {msg.content}")
print("End of conversation.")

In [ ]:
for msg in messages:
    if isinstance(msg, AIMessage):
        if "Tool Name:" in msg.content:
            break
        else:
            print(f"ASSISTANT : {msg.content}")
    elif isinstance(msg, HumanMessage):
        print(f"YOU : {msg.content}")

print("ASSISTANT :\n")

for items in tools_suggestion.tools:
    search_query="Can you provide me the purchase link of the "+ items.tool_name+" from Amazon?"
    results=search.invoke(search_query)
    if len(results):
        print(f"Tool name : {items.tool_name}\nTool description : {items.description}\nLink Title: {results[0].get('title')}\nURL: {results[0].get('url')}\n\n")


for msg in messages:
    if isinstance(msg, HumanMessage):
        print(f"YOU : {msg.content}")
    else:
        if isinstance(msg, AIMessage):
            print(f"ASSISTANT : {msg.content}")

ASSISTANT : How can I assist you today?
YOU : I want to hang a wall mirror in my living room.
ASSISTANT : What problems are you running into with hanging the wall mirror?
YOU : I'm trying to hang a wall mirror but I'm not sure how to do it properly.
ASSISTANT : Describe it in more detail
YOU : I don't want to scratch the walls or break the glass. It has two hooks for attachments.
ASSISTANT : Do you have experience hanging a mirror?
YOU : Yes, I have hang a small to medium-sized mirror before.
ASSISTANT : Could you provide with a list of the tools and materials you have?
YOU : I have a hammer, nails, and a level.
ASSISTANT : Here are some questions to help me understand your task better:
ASSISTANT : Q1: What type of wall are you hanging the mirror on?
Options:
Drywall
Plaster
Brick / Concrete
I'm not sure

YOU : Drywall
ASSISTANT : Q2: What is the approximate weight of the mirror?
Options:
Light (under 10 lbs / 5 kg)
Medium (10-25 lbs / 5-11 kg)
Heavy (over 25 lbs / 11 kg)

YOU : Heavy 

In [94]:
with open("conversation.txt", "w") as f:
    for msg in messages:
        if isinstance(msg, HumanMessage):
            f.write(f"YOU : {msg.content}\n")
        else:
            if isinstance(msg, AIMessage):
                f.write(f"ASSISTANT : {msg.content}\n")
    f.write("End of conversation.\n")